In [1]:
# Dependancies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
# import requests
# import pymongo

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\chris\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Step 1 - Scraping

## NASA Mars News

In [3]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [4]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [5]:
element = news_soup.select_one('div.list_text')

In [6]:
news_title = element.find('div',class_='content_title').get_text()
news_title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

In [7]:
news_p = element.find('div',class_='article_teaser_body').get_text()
news_p

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

## JPL Mars Space Images - Featured Image

In [8]:
img_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(img_url)

In [9]:
img_button = browser.find_by_css('button.btn.btn-outline-light')
# browser.find_by_text("Full Image")
# browser.find_by_css("")
img_button.click()

In [10]:
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [11]:
full_img_url = img_soup.find('img', class_='fancybox-image').get('src')
full_img_url

'image/featured/mars3.jpg'

In [12]:
full_img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{full_img_url}'
full_img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

## Mars Facts

In [13]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [14]:
mars_facts = pd.read_html(facts_url)[0]
mars_facts = mars_facts.rename(columns={0:'Measure',1:'Value'})
mars_facts.set_index('Measure')

,Value
Measure,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
mars_facts = mars_facts.to_html()
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measure</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</

## Mars Hemispheres

In [16]:
base_url = 'https://astrogeology.usgs.gov'
mars_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_url)

html = browser.html
mars_soup = BeautifulSoup(html, 'html.parser')

In [17]:
# List all hemisphere items
itemLink_ls = mars_soup.find_all('div',class_='description')

hemisphere_image_urls = []

itemLink_url = ''
title = ''
full_img_url = ''

# Loop through hemisphere items
for item in itemLink_ls:
    if (item.a):
        itemLink_url = item.a.get('href')
        itemLink_url = f'{base_url}{itemLink_url}'
        if (item.a.h3):
            title = item.a.h3.get_text()
    
    # Get full image url
    browser.visit(itemLink_url)
    html = browser.html
    hem_soup = BeautifulSoup(html, 'html.parser')
    
    x = hem_soup.find('div', class_='downloads')
    if (x.ul):
        if (x.li):
            if (x.a):
                full_img_url = x.ul.li.a.get('href')
    
    # Apend to list
    hemisphere_image_urls.append({'Title':title, 'Image_URL':full_img_url})
    
    # Reset variables
    itemLink_url = ''
    title = ''
    full_img_url = ''   
    
hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image_URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image_URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image_URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image_URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# Step 2 - MongoDB and Flask Application

# Step 3 - Submission